# Stream Cipher

A stream cipher is a symmetric key cipher where the plaintext is encrypted one digit at a time. Encryption is achieved by xoring the plaintext with a stream of pseudorandom digits obtained as an expansion of the key.

![stream-cipher](stream-cipher.png)

In [ ]:
# use the class Bits of the third-part library bitstring
# for the conversion from string to bit-stream
from bitstring import Bits
# use an LFSR as pseudo random number generator (PRNG)
from lfsr import LFSR

# message that Alice sends to Bob
message = 'hello world!'

# define two PRNG initialized with the same key
poly, key = [64, 4, 3, 1, 0], 0x0123456789ABCDEF
prngA = LFSR(poly, state=key) # Alice
prngB = LFSR(poly, state=key) # Bob

# convert message to bits
plaintextA = Bits(bytes=bytes(message, 'utf-8'))
# message encryption
ciphertext = Bits([b^s for b, s in zip(plaintextA, prngA)])
# message decryption
plaintextB = Bits([e^s for e, s in zip(ciphertext, prngB)])

# print
print_text = lambda b: '0x{} "{}"'.format(
    b.hex, b.bytes.decode('utf-8', 'replace'))
print('plaintextA:', print_text(plaintextA))
print('ciphertext:', print_text(ciphertext))
print('plaintextB:', print_text(plaintextB))
        

## Task1: Stream Cipher

> Define an object representing a Stream Cipher that, given a PRNG and a key, is able to encrypt and decrypt a message one bit at a time.

**Inputs**:
- **key**: `int` representing the shared secret key.
- **PRNG**: (optional, default None) Iterator implementing a PRNG that produce a pseudorandom bit stream starting from an initial seed. If None an LFSR is used as PRNG.

**Methods**:
- `encrypt`: encrypts a plaintext (`str` or `bytes`) and returns the corresponding cyphertext (`bytes`);
- `decrypt`: decrypts a cypertext (`str` or `bytes`) and returns the corresponding plaintext (`bytes`);

**Template**:
```python
class StreamCipher(object):
    ''' class docstring '''

    def __init__(self, key, prng=None):
        ''' constructor docstring '''
        # do stuff
        self.prng = ... 

    def encrypt(self, plaintext): 
        # do stuff
        return ciphertext

    def decrypt(self, ciphertext):
        # do stuff
        return plaintext
```

In [ ]:
class StreamCipher(object):
    '''
    Stream cipher.

    Methods
    -------
    encrypt(self, plaintext):
        encrypt a plaintext
    decrypt(self, ciphertext): 
        decrypt a ciphertext
    '''
    def __init__(self, key, prng=None, **kwargs):        
        '''
        Parameters
        ----------
        key: int,
            secret key for PRNG initialization
        prng: class, optional (default None),
            pseudo-random-number-generator (PRNG) for the generation of
            the random bit-stream used for encryption and decryption
        kwargs: dict,
            keyword arguments for `prng`
        '''
        if prng is None:
            poly = [64, 4, 3, 1, 0]
            state = key if key > 0 else None
            self.prng = LFSR(poly, state=state)
        else:
            self.prng = prng(key, **kwargs)
        
    def encrypt(self, plaintext):
        '''encrypt a `plaintext` (str, bytes, bitstring.Bits) and 
        return the corresponding cyphertext (bytes) '''
        return self._crypt(plaintext)
    
    def decrypt(self, ciphertext):
        ''' decrypt a `cypertext` (str, bytes, bitstring.Bits) and 
        return the corresponding plaintext (bytes) '''
        return self._crypt(ciphertext)
    
    def _crypt(self, text):
        if isinstance(text, Bits):
            bits = text
        if isinstance(text, bytes):
            bits = Bits(bytes=text)
        elif isinstance(text, str):
            bits = Bits(bytes=text.encode('utf-8'))
        else:
            raise TypeError('input type is not supported')
        return Bits([b^s for b, s in zip(bits, self.prng)]).bytes
        

In [ ]:
message = 'hello world!'
key = 0x0123456789ABCDEF 

# define a function that create an instance of an LFSR
prng = lambda key: LFSR([64, 4, 3, 1, 0], state=key)

# create a StreamCipher instance for both Alice and Bob
alice = StreamCipher(key, prng=prng) 
bob   = StreamCipher(key, prng=prng)

plaintextA = message.encode('utf-8')   # string to bytes 
ciphertext = alice.encrypt(plaintextA) # encryption by Alice
plaintextB = bob.decrypt(ciphertext)   # decryption by Bob

print(plaintextA) # -> b'hello world!' 
print(ciphertext) # -> b'\x87\x02\xc7O\xa2e\xfen\x14\xfb\xafv' 
print(plaintextB) # -> b'hello world!' 


## Task2: A5/1

> Define an iterator that implements a PRNG based on the A5/1 architecture and use it in a `StreamCipher` instance to encrypt/decrypt a message.

A5/1 is a stream cipher used to provide privacy in the GSM cellular telephone standard.

It is based on a combination of three LFSRs with irregular clocking. At each cycle, the clocking bit of all three registers is examined and the majority bit is determined. A register is clocked if the clocking bit agrees with the majority bit.

| LFSR | length | &nbsp; &nbsp; Feedback Polynomial &nbsp; &nbsp; | clocking bit |
|:----:|:------:|:-----------------------------------------------:|:------------:|
|   1  |   19   |      \\( x^{19}+x^{18}+x^{17}+x^{14}+1 \\)      |      10      |
|   2  |   22   |      \\(               x^{22}+x^{21}+1 \\)      |      11      |
|   3  |   23   |      \\(    x^{23}+x^{22}+x^{21}+x^8+1 \\)      |      12      |

![A5/1](A5_1.png)


**Initialization**

A5/1 is initialised using a 64-bit private key together with a publicly known 22-bit frame number.
- Initially, the registers are set to zero. 
- The 64-bit secret key is mixed: in cycle $i$ (with 0≤$i$<64), the $i$-th key bit is added to the most significant bit of each register using XOR.
- Similarly, the 22-bits frame number is added in 22 cycles.

Then, the cipher is clocked using the normal majority clocking mechanism for 100 cycles, with the output discarded.

Finally, the cipher is ready to produce two 114 bit sequences of output keystream, first 114 for downlink, last 114 for uplink.

In [ ]:
from functools import reduce
from operator import xor

from lfsr import LFSR, reverse

In [ ]:

class A5_1(object):
    '''
    A5/1 stream cipher.
    
    Attributes
    ----------
    output: bool,
        stream cipher output bit.
    LFSRs: lfsr.LFSR,
        linear feedback shift registers that compose the A5/1 stream cipher
    majority: bool,
        majority bit
    '''
    
    def __init__(self, key, frame=0, verbose=False):
        '''
        key: int, list/string of 0/1, bytes, 
            64-bit key, bytes are considered little endian.
        frame: int or list/string of 0/1, 
            22-bit frame
        '''
        key_length = 64
        frame_length = 22
        polys = [
            [19, 18, 17, 14, 0],
            [22, 21, 0],
            [23, 22, 21, 8, 0],
        ]
                
        if isinstance(key, (bytes, bytearray)):
            key = int.from_bytes(key, byteorder='little')#wrong
        if isinstance(key, int):
            key_str = '{:0{}b}'.format(
                key & ((1 << key_length)-1), key_length)[::-1]
            key = [bool(int(b)) for b in key_str]
        elif (not hasattr(key, '__iter__')) \
            or (not len(key) == key_length) \
            or (not all(int(b) in (0, 1) for b in key)):
            raise TypeError('input type is not supported')
            
        if type(frame) is int:
            frame_string = '{:0{}b}'.format(
                frame & ((1 << frame_length)-1), frame_length)[::-1]
            frame = [bool(int(b)) for b in frame_string]
        elif (not hasattr(frame, '__iter__')) \
            or (not len(frame) == frame_length) \
            or (not all(int(b) in (0, 1) for b in frame)):
            raise TypeError('input type is not supported')
        
        self.vebose = verbose
        self._LFSRs = [LFSR(poly, state=0) for poly in polys]
        self._ckbits = [10, 11, 12]
        self._output = self._compute_output()
        self._majority = self._compute_majority()
        self._count = 0
        
            
        # LFSRs initialization
        if verbose: 
            header = ' iter  LFSR1  LFSR2  LFSR3 maj out'
            print('--- key insertion ----')
            print(''.join([str(int(b)) for b in key]))
            print(header)
        for bit in key:
            self._insert_bit(bit)
        self._count = 0
            
        if verbose: 
            print('--- frame insertion ----')
            print(''.join([str(int(b)) for b in frame]))
            print(header)
        for bit in frame:
            self._insert_bit(bit)
        self._count = 0
            
        if verbose: print('--- key mixing ----')
        if verbose: print(header)
        for _ in range(100): 
            next(self)
        self._count = 0
            
        if verbose: print('--- stream cipher ----')
        if verbose: print(header)
            

    @property
    def LFSRs(self):
        return self._LFSRs
    
    @LFSRs.setter
    def LFSRs(self, val):
        raise AttributeError('Denied')

    @property
    def output(self):
        return self._output
    
    @output.setter
    def output(self, val):
        raise AttributeError('Denied')

    @property
    def majority(self):
        return self._majority
    
    @majority.setter
    def majority(self, val):
        raise AttributeError('Denied')
        
    def __iter__(self):
        return self
        
    def __next__(self):
        ''' clock stream cipher and returns the outputbit '''
        # clock LFSR_i if ckbit_i is equal to majority bit
        for lfsr, ckbit in zip(self.LFSRs, self._ckbits):
            if bool(lfsr.state & (1 << ckbit)) == self._majority:
                next(lfsr)
        # update output and majority bit
        self._output = self._compute_output()
        self._majority = self._compute_majority()
        self._count += 1
        self._log()
        
        return self._output  
    
    def _insert_bit(self, bit):
        ''' Insert `bit` (bool) in each LFSR '''
        # every LFSR is clocked independently from majority bit
        for lfsr in self.LFSRs:
            lfsr.feedback ^= bit
            next(lfsr)
        # update output and majority bit
        self._output = self._compute_output()
        self._majority = self._compute_majority()
        self._count += 1
        self._log()
        
    def _compute_majority(self):
        ''' Compute majority bit '''
        # get the values of ckbits for each LFSR
        bits = [bool(lfsr.state & (1 << bit))
               for lfsr, bit in zip(self.LFSRs, self._ckbits)]
        # compute majority
        majority = sum(bits) > (len(bits)//2)
        return majority
        
    def _compute_output(self):
        ''' Compute output bit '''
        output = reduce(xor, [lfsr.output for lfsr in self.LFSRs])
        return output
    
    def __str__(self):
        _str = 'state: ({}), majority: {}, output: {}'.format(
            ', '.join(['0x{:0{}x}'.format(lfsr.state, 1+len(lfsr)//4) 
                       for lfsr in self.LFSRs]),
            int(self._majority),
            int(self._output),
        )
        return _str
        
    def __repr__(self):
        return 'A5/1({})'.format(self.__str__())
    
    def _log(self):
        if self.vebose:
            print(' {:4d}  {}  {}   {}'.format(
                self._count,
                ' '.join(['{:0{}x}'.format(
                            lfsr.state, #reverse(lfsr.state, len(lfsr)), 
                            1+len(lfsr)//4)
                          for lfsr in self.LFSRs]),
                int(self._majority),
                int(self._output),
            ))
        

In [ ]:
key, frame = 0x0123456789ABCDEF, 0x2F695A # my key
a51 = A5_1(key, frame=frame, verbose=True)

In [ ]:
message = 'hello world!'
key, frame = 0x0123456789ABCDEF, 0x2F695A # my key

# create a StreamCipher instance for both Alice and Bob
alice = StreamCipher(key, prng=A5_1, frame=frame)
bob = StreamCipher(key, prng=A5_1, frame=frame)

plaintextA = message.encode('utf-8')
ciphertext = alice.encrypt(plaintextA)
plaintextB = bob.decrypt(ciphertext)
print('plaintextA:', plaintextA, plaintextA.decode('utf-8', 'replace'))
print('ciphertext:', ciphertext, ciphertext.decode('utf-8', 'replace'))
print('plaintextB:', plaintextB, plaintextB.decode('utf-8', 'replace'))

## Task3: Stream Cipher (2)

> Add to the `StreamCipher` class the possibility to encrypt/decrypt a message using bytes as digits instead of bits.

**Inputs**:
- **key**: `int` representing the shared secret key.
- **PRNG**: (optional, default None) Iterator implementing a PRNG that produce a pseudorandom *digit* stream starting from an initial seed. *A digit can be either a bit or a byte depending on the value of the digit parameter*. If None an LFSR is used as PRNG.
- **digit**: (optional, default "bit") `str` that determines the type of bigit: "bit" or "byte".

**Methods**:
- `encrypt`: encrypts a plaintext (`str` or `bytes`) and returns the corresponding cyphertext (`bytes`);
- `decrypt`: decrypts a cypertext (`str` or `bytes`) and returns the corresponding plaintext (`bytes`);

**Template**:
```python
class StreamCipher(object):
    ''' class docstring '''

    def __init__(self, key, prng=None, digit='bit'):
        ''' constructor docstring '''
        # do stuff
        self.prng = ... 

    def encrypt(self, plaintext): 
        # do stuff
        return ciphertext

    def decrypt(self, ciphertext):
        # do stuff
        return plaintext
```

In [ ]:
# message that Alice sends to Bob
message = 'hello world!'
key = 0x0123456789ABCDEF

# define a function that generate a random byte
def prng(seed):
    import random
    _prng = random.Random(seed)
    while True:
        yield _prng.randint(0, 255)

# define two PRNG initialized with the same key
prngA = prng(key) # Alice
prngB = prng(key) # Bob

# convert message to bits
plaintextA = bytes(message, 'utf-8')
# message encryption
ciphertext = bytes([b^s for b, s in zip(plaintextA, prngA)])
# message decryption
plaintextB = bytes([e^s for e, s in zip(ciphertext, prngB)])

# print
print_text = lambda B: '0x{} "{}"'.format(
    int.from_bytes(B, byteorder='little'), B)
print('plaintextA:', print_text(plaintextA))
print('ciphertext:', print_text(ciphertext))
print('plaintextB:', print_text(plaintextB))
        

In [ ]:
class StreamCipher(object):
    '''
    Stream cipher.

    Methods
    -------
    encrypt(self, plaintext):
        encrypt a plaintext
    decrypt(self, ciphertext): 
        decrypt a ciphertext
    '''
    def __init__(self, key, prng=None, digit='bit', **kwargs):        
        '''
        Parameters
        ----------
        key: int,
            secret key for PRNG initialization
        prng: class, optional (default None),
            pseudo-random-number-generator (PRNG) for the generation of
            the random bit-stream used for encryption and decryption
        kwargs: dict,
            keyword arguments for `prng`
        '''
        if prng is None:
            poly = [64, 4, 3, 1, 0]
            state = key if key > 0 else None
            self.prng = LFSR(poly, state=state)
            self._crypt = self._crypt_bit 
        else:
            if digit == 'bit':
                self._crypt = self._crypt_bit
            elif digit == 'byte':
                self._crypt = self._crypt_byte
            else:
                raise ValueError('digit not valid')
            self.prng = prng(key, **kwargs)
        
    def encrypt(self, plaintext):
        '''encrypt a `plaintext` (str, bytes, bitstring.Bits) and 
        return the corresponding cyphertext (bytes) '''
        return self._crypt(plaintext)
    
    def decrypt(self, ciphertext):
        ''' decrypt a `cypertext` (str, bytes, bitstring.Bits) and 
        return the corresponding plaintext (bytes) '''
        return self._crypt(ciphertext)
    
    def _crypt_bit(self, text):
        if isinstance(text, Bits):
            bits = text
        if isinstance(text, bytes):
            bits = Bits(bytes=text)
        elif isinstance(text, str):
            bits = Bits(bytes=text.encode('utf-8'))
        else:
            raise TypeError('input type is not supported')
        return Bits([b^s for b, s in zip(bits, self.prng)]).bytes
    
    def _crypt_byte(self, text):
        if isinstance(text, Bits):
            _bytes = text.bytes
        if isinstance(text, bytes):
            _bytes = text
        elif isinstance(text, str):
            _bytes = text.encode('utf-8')
        else:
            raise TypeError('input type is not supported')
        return bytes([b^s for b, s in zip(_bytes, self.prng)])
        

In [ ]:
message = 'hello world!'
key = 0x0123456789ABCDEF

# define a function that generate a random byte
def prng(seed):
    import random
    _prng = random.Random(seed)
    while True:
        yield _prng.randint(0, 255)
        
# create a StreamCipher instance for both Alice and Bob
alice = StreamCipher(key, prng=prng, digit='byte') 
bob   = StreamCipher(key, prng=prng, digit='byte')

plaintextA = message.encode('utf-8')   # string to bytes 
ciphertext = alice.encrypt(plaintextA) # encryption by Alice
plaintextB = bob.decrypt(ciphertext)   # decryption by Bob

print(plaintextA) # -> b'hello world!' 
print(ciphertext) # -> b'\x87\x02\xc7O\xa2e\xfen\x14\xfb\xafv' 
print(plaintextB) # -> b'hello world!' 


## Task4: Rivest Cipher 4 (RC4)

> Define an iterator that implements a PRNG based on the RC4 architecture and use it in the new `StreamCipher` class to encrypt/decrypt a message.

RC4 is a stream cipher that generates the keystream from a secret internal state which consists of two parts:
- A permutation $P$ of all 256 possible bytes.
- Two 8-bit index-pointers (denoted $i$ and $j$).

$P$ is initialized with a variable length key by means of the *key-scheduling algorithm* (**KSA**).

Then, the keystream is generated using the *pseudo-random generation algorithm* (**PRGA**) that updates the indexes $i$ and $j$, modifies the permutation $P$ and generates a random byte.


**Key Scheduling Algorithm (KSA)**

The KSA is used to initialize the permutation $P$ starting from a key composed by 𝐿 bytes. Typical values for 𝐿 range from 40 to 256.

- $P$ is initialized with an identity permutation ($P[i]=i$).
- Then, bytes of $P$ are mixed iteratively in a way that depends on the key.

**Pseudocode**:
> **Input** \\( k = [k_0, k_1, \dots, k_{L-1}]\\), with \\( k_i \in {0, 1, \dots, 255} \\) <br>
> \\( j \leftarrow 0 \\) <br>
> **for** \\( i = 0, 1, \dots, 255 \\) <br>
> \\( \qquad P[i] \leftarrow i \\) <br>
> **endfor** <br>
> **for** \\( i = 0, 1, \dots, 255 \\) <br>
> \\( \qquad j \leftarrow (j + P[i] + k[i \\) mod \\( L]) \\) mod \\( 256 \\) <br>
> \\( \qquad P[i], P[j] \leftarrow P[j], P[i] \\) <br>
> **endfor** <br>
> **Output** \\( P \\) <br>

**Pseudo-random generation Algorithm (PRGA)**

For each iteration, PRGA modifies the state (represented by the permutation $P$ and the pair of indexes $i$, $j$) and outputs a byte.

In each iteration:
- $i$ is incremented, 
- $j$ is updated by adding the value $P[i]$, 
- $P[i]$ and $P[j]$ are swapped.
- the output byte is element of $P$ ant the location $P[i]+P[j]$  (mod 256)


**Pseudocode**:
> **State** \\( P, i, j \\) <br>
> \\( i \leftarrow (i + 1) \\) mod \\( 256 \\) <br>
> \\( j \leftarrow (j + P[i]) \\) mod \\( 256 \\) <br>
> \\(  P[i], P[j] \leftarrow P[j], P[i] \\) <br>
> \\( K \leftarrow P[(P[j] + P[i]) \\) mod \\( 256] \\) <br>
> **Output** \\( K \\) <br>

**RC4-drop[$n$]**

RC4 has many known vulnerabilities mainly related to the correlation between the key and the first bytes of the permutation $P$. 
Most of them can be avoided by discarding the first 𝑛 bytes of the output stream, from where it becomes RC4-drop[$n$].
Typical values for $n$ are:
- $n = 768$
- $n = 3072$ (more conservative value)


In [ ]:
class RC4(object):
    '''
    Rivest Cipher 4 (RC4) Stream Cipher.
    
    Attributes
    ----------
    P: list of int,
        RC4 Permutation
    i, j: int,
        RC4 indices
    '''
    
    def __init__(self, key, key_length=None, drop=0):
        ''' 
        Parameters
        ----------
        key: int, bytes, str, list of int,
            secret key
        key_length: bytes, optional (default None)
            number of bytes composing the key in case key is an int.
        drop: int, optional (default 0)
            number of output bytes to drop after initialization
        '''    
        # check parameters
        if isinstance(key, int):
            key = int(key).to_bytes(byteorder='little', length=key_length)
        elif isinstance(key, (bytes, bytearray))\
            or (hasattr(key, '__iter__') \
                and all(isinstance(B, int) for B in key)):
            key = bytes(key)
        elif isinstance(key, str):
            key = key.encode('utf-8')
        else:
            raise TypeError('key format is not supported')
            
        if not isinstance(drop, int) or drop < 0:
            raise ValueError('drop must be an integer >= 0')
        
        # initialization
        self.P = self.ksa(key)
        self.i, self.j = 0, 0
        
        # dropping output bytes for safety issues
        for _, byte in zip(range(drop), self):
            pass
    
    @staticmethod
    def ksa(key):
        ''' returns the permutation P initialized by means of the 
        Key-scheduling Algorithm (KSA) with the input key (bytes)
        '''
        P = list(range(256)) # identity permutation
        j = 0
        for i in range(256):
            j = (j + P[i] + key[i % len(key)]) % 256
            P[i], P[j] = P[j], P[i]
        return P
    
    def prga(self):
        ''' Pseudo-random generation algorithm (PRGA) that generates a random
        byte starting from the Permutation P and the pair of indexes i, j.
        '''
        self.i = (self.i + 1) % 256
        self.j = (self.j + self.P[self.i]) % 256
        self.P[self.i], self.P[self.j] = self.P[self.j], self.P[self.i]
        byte = self.P[(self.P[self.i] + self.P[self.j]) % 256]
        return byte
        
    def __iter__(self):
        return self
        
    def __next__(self):
        return self.prga() 
    
    def __str__(self):
        _str = 'i: 0x{:02x}, j: 0x{:02x}, P: 0x{:0512x}'.format(
            self.i, self.j, int.from_bytes(bytes(self.P), byteorder='little')
        )
        return _str
    
    def __repr__(self):
        return 'RC4({})'.format(self.__str__())
        

In [ ]:
message = 'hello world!'
key = b'0123456789ABCDEF'
ndrop = 3072
        
# create a StreamCipher instance for both Alice and Bob
alice = StreamCipher(key, digit='byte', prng=RC4, drop=ndrop) 
bob   = StreamCipher(key, digit='byte', prng=RC4, drop=ndrop)

plaintextA = message.encode('utf-8')   # string to bytes 
ciphertext = alice.encrypt(plaintextA) # encryption by Alice
plaintextB = bob.decrypt(ciphertext)   # decryption by Bob

print(plaintextA) # -> b'hello world!' 
print(ciphertext) # -> b'/\x9e\xf9\x83@\x81}\xa9\xd0\xd4\xd5\xf4' 
print(plaintextB) # -> b'hello world!' 
